In [ ]:
import numpy as np
import pandas as pd
import math
import random
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
num_actions=3  #0 for hold, 1 for buy, 2 for sell
def create_model():
        model = Sequential()
        model.add(Dense(units=32, input_shape=(20,) , activation="relu"))
        model.add(Dense(units=16, activation="relu"))
        model.add(Dense(units=8, activation="relu"))
        model.add(Dense(num_actions, activation="linear"))
        model.compile(loss="mse", optimizer=Adam(learning_rate=0.0025),metrics=['accuracy'])
        return model
class EpsilonGreedyStrategy():
    def __init__(self,epsilon,epo_min, decay):
        self.epsilon = epsilon
        self.epo_min = epo_min
        self.decay = decay  
    def get_exploration_rate(self, current_step):
        return self.epo_min+(self.epsilon-self.epo_min)*math.exp(-1*current_step*self.decay) 
def getStockData(key):
    stocks = []
    lines = open(key+".csv","r").read().splitlines()
    for line in lines[1:]:
        stocks.append(float(line.split(",")[4])) 
    return stocks    
def sigmoid(x):
    return 1/(1+math.exp(-x))    
def getState(data, t, n):
    d =t-n+1
    block=data[d:t+1]
    return np.array(block)
    # res=[]
    # for i in range(n-1):
    #   res.append((block[i+1]-block[i]))  
    # return np.array(res)
    

        

In [ ]:
#random.seed(2)
gamma = 0.9  # Discount factor for past rewards
epsilon = 1.0  # Epsilon greedy parameter
epsilon_min = 0.1  # Minimum epsilon greedy parameter
decay_rate = 0.001  #epsilon decay rate  
memory_size = 100000
num_episodes = 50
num_space=[i for i in range(num_actions)]
action_data = []
state_data = []
next_state_data = []
rewards_data = []
done_data = []
stock_name = input("Enter stock_name : ")
stock_name = str(stock_name)
window_size = 20
data=getStockData(stock_name)
l=len(data)-1
batch_size = 32
i=window_size-1
update_target_network = 30 #update target network weights after 30 days 
strategy=EpsilonGreedyStrategy(epsilon,epsilon_min,decay_rate)
# The first model makes the predictions for Q-values which are used to
# make a action.
model = create_model()
# Target model used to predict optimal Q-values
target_model= create_model()
loss_function = keras.losses.Huber()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
MAX_TRANSACTIONS =50   #maximum number of time we can do a buy after a selling
TOTAL_MONEY = int(input())    #total money that we have in cash(virtual )
money=TOTAL_MONEY
for epi in range(100):
    TOTAL_MONEY=money
    CURRENT_STOCKS_BOUGHT = 0      #the number of stocks we have bought currently
    CURRENT_TRANSACTION_COUNT = 0
    state = getState(data, i, window_size)
    total_profit = 0
    for day in range(i,l):
        action=0 
        done = True if day == l - 1 else False 
        epsilon=strategy.get_exploration_rate(day-i+1)
        if random.random()<= epsilon:
            action= np.random.choice(num_space)
        else: 
            state_tensor = tf.convert_to_tensor(state)
            state_tensor = tf.expand_dims(state_tensor, 0)
            action= tf.argmax((model(state_tensor,training=False))[0]).numpy()
            
        next_state =getState(data, i+1, window_size) 
        
        reward=0
        x=0
        if CURRENT_TRANSACTION_COUNT<MAX_TRANSACTIONS:
            x=TOTAL_MONEY/(MAX_TRANSACTIONS- CURRENT_TRANSACTION_COUNT) 
            x=round(x*100)/100      
        if( action==1 and x>=data[day]):
            y=int(x/data[day])
            TOTAL_MONEY-=x
            reward=(data[day+1]-data[day])*y
            CURRENT_STOCKS_BOUGHT+=y
            CURRENT_TRANSACTION_COUNT+=1  
        elif action ==2 and CURRENT_STOCKS_BOUGHT>0:
            reward=data[day]*CURRENT_STOCKS_BOUGHT + TOTAL_MONEY - money    
            TOTAL_MONEY+= data[day]*CURRENT_STOCKS_BOUGHT
            CURRENT_TRANSACTION_COUNT =0
            CURRENT_STOCKS_BOUGHT=0
        #elif action ==0:
            # if(data[day]>data[day+1] and CURRENT_STOCKS_BOUGHT):
            #     reward=-(data[day]*CURRENT_STOCKS_BOUGHT + TOTAL_MONEY - money)
            # elif data[day]<data[day+1] and x>=data[day]:
            #     stocks=int(x/data[day])                
            #     reward=-(data[day+1]-data[day])*stocks
        else:
            action=0                  
                           
        reward=round(reward*100)/100
        TOTAL_MONEY=round(TOTAL_MONEY*100)/100;
        
        action_data.append(action)
        state_data.append(state)
        next_state_data.append(next_state)
        done_data.append(done)
        rewards_data.append(reward)
        state = next_state    
        
        #Experience replay
        if len(done_data) > batch_size:
            sample = np.random.choice(range(len(done_data)), size=batch_size)

            state_sample = np.array([state_data[i] for i in sample])
            state_next_sample = np.array([next_state_data[i] for i in sample])
            rewards_sample = [rewards_data[i] for i in sample]
            action_sample = [action_data[i] for i in sample]  
            done_sample=[done_data[i] for i in sample]
          
            future_q_val= target_model.predict(state_next_sample)
            target_q_val= np.array(rewards_sample) + gamma* tf.reduce_max(future_q_val, axis=1)
            masks = tf.one_hot(action_sample, num_actions)
            

            with tf.GradientTape() as tape:
               
                cur_q_values = model(state_sample)

                # Apply the masks to the Q-values to get the Q-value for action taken
                cur_q_action = tf.reduce_sum(tf.multiply(cur_q_values, masks), axis=1)
                # Calculate loss between Target Q-value and current Q-value
                loss = loss_function(target_q_val, cur_q_action)

            grad = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grad, model.trainable_variables))
               
                                         
        if  day-i+1 % update_target_network == 0:
            # update the the target network with new weights
            target_model.set_weights(model.get_weights()) 
             

        if done:
            break
        if len(rewards_data) >memory_size:
            rewards_data[len(rewards_data)%memory_size]=reward
            state_data[len(rewards_data)%memory_size]=state
            next_state_data[len(rewards_data)%memory_size]=next_state
            action_data[len(rewards_data)%memory_size]=action
            done_data[len(rewards_data)%memory_size]=done                

model.save("mymodel")  

            
    
            

          
        


Enter stock_name : CSCO
20000
INFO:tensorflow:Assets written to: mymodel/assets


Test evaluation

In [ ]:
stock_name = input("Enter Stock_name: ")
data = getStockData(stock_name)
l = len(data) - 1
print(l)
total_profit = 0
window_size=20
i=window_size-1
MAX_TRANSACTIONS = 50   #maximum number of time we can do a buy after a selling
TOTAL_MONEY = int(input())    #total money that we have in cash(virtual )
CURRENT_STOCKS_BOUGHT = 0      #the number of stocks we have bought currently
CURRENT_TRANSACTION_COUNT = 0
money=TOTAL_MONEY
state = getState(data, i, window_size)
model=keras.models.load_model("mymodel")
total_days=l-i+1
for day in range(i,l):
     
    action =0  
    done = True if day == l - 1 else False 
  
    state_tensor = tf.convert_to_tensor(state)
    state_tensor = tf.expand_dims(state_tensor, 0)
    action= tf.argmax((model(state_tensor))[0]).numpy()
   
    next_state = getState(data, day + 1, window_size)
        
    if day == l-1:
       action =2

    if(action==1 and TOTAL_MONEY>=data[day] and CURRENT_TRANSACTION_COUNT<MAX_TRANSACTIONS):
        x=TOTAL_MONEY/(MAX_TRANSACTIONS- CURRENT_TRANSACTION_COUNT) 
        x=round(x*100)/100
        y=int(x/data[day])
        TOTAL_MONEY-=data[day]*y;
        CURRENT_STOCKS_BOUGHT+=y
        CURRENT_TRANSACTION_COUNT+=1  
    elif action ==2 and CURRENT_STOCKS_BOUGHT>0:
        TOTAL_MONEY+= data[day]*CURRENT_STOCKS_BOUGHT
        total_profit+=TOTAL_MONEY - money     
        CURRENT_TRANSACTION_COUNT =0
        CURRENT_STOCKS_BOUGHT=0
    else:
        action =0    
      
    TOTAL_MONEY=round(TOTAL_MONEY*100)/100
    total_profit=round(total_profit*100)/100
    
    if done:
        print("Rs."+ "{0:.2f}".format(total_profit))
        break       

Enter Stock_name: CSCO_TESTDATA
501
20000
Rs.2300.69
